In [16]:
import re
import string
import numpy as np
import pandas as pd
import textdistance
from collections import Counter

#import library <br>
that are needed to run this project<br><br>

run all this in your terminal if you don't have any library:<br>
    > pip install re<br>
    > pip install numpy<br>
    > pip install pandas<br>
    > pip install textdistance<br>

In [17]:
def process_data(file_name):
    words = []
    with open(file_name, "r") as f:
        data = f.read()
        data_lower = data.lower()
        words = re.findall(r"\w+", data_lower)
    return words

def process_freq(file_name):
    freqs = []
    with open(file_name, "r") as f:
        data = f.read()
        freqs = list(map(int, re.findall(r"\w+", data)))
    return freqs

In [18]:
def save_vocab():
    word_l = process_data("words4.txt")
    vocab = set(word_l)
    print(f"The first 10 words in the text are : \n{word_l[0:10]}")
    print(f"There are {len(vocab)} words in the vocabulary.\n")
    return word_l

def get_count(word_l, word, freq=[]):
    word_count_dict = {}
    for i in range(len(word_l)):
        word_count_dict[word_l[i]] = 1
    word_count_dict = Counter(word_l)
    print(f"There are {len(word_count_dict)} key values pairs")
    print(f"The count for the word {word} is {word_count_dict.get(word, 0)}")
    return word_count_dict

In [19]:
def get_probabilities(word_count_dict):
    probabilities = {}
    total = sum(word_count_dict.values())
    for i in word_count_dict:
        probabilities[i] = float("{:f}".format(word_count_dict[i] / total))
    print(f"Length of probabilities is {len(probabilities)}")
    # print(f"P(\"{word}\") is {probabilities[word]:.4f}\n")
    return probabilities

In [20]:
word = input("Enter word : ")

word_l = save_vocab()
# freqs = process_freq("freq.txt")
word_count_dict = get_count(word_l, word)
probabilities = get_probabilities(word_count_dict)

The first 10 words in the text are : 
['the', 'of', 'and', 'to', 'a', 'in', 'for', 'is', 'on', 'that']
There are 10000 words in the vocabulary.

There are 10000 key values pairs
The count for the word linaar is 0
Length of probabilities is 10000


In [21]:
def delete_letter(word, verbose=False):
    delete_l = []
    delete_l = [word[0:i] + word[i+1 : len(word)] for i in range(len(word))]
    if verbose:
        print(f"input word = {word}, delete_l = {delete_l}\n")
    return delete_l

In [22]:
def switch_letter(word, verbose=False):
    switch_l = []
    word1 = list(word)
    for i in range(len(word1)-1):
        a1 = list(word1)
        a1[i], a1[i+1] = a1[i+1], a1[i]
        b = "".join(a1)
        switch_l.append(b)
    if word in switch_l:
        switch_l.remove(word)
    if verbose:
        print(f"Input word = {word}, switch_l = {switch_l}\n")
    return switch_l

In [23]:
def replace_letter(word, verbose=False):
    replace_l = []
    replace_set = []
    lower = string.ascii_lowercase
    for i in range(len(word)):
        temp = [word[0:i] + j + word[i+1:len(word)] for j in lower]
        temp.remove(word)
        replace_set.extend(temp)
    replace_l = sorted(list(replace_set))
    if verbose:
        print(f"Input word = {word}, replace_l = {replace_l}\n")
    return replace_l

In [24]:
def insert_letter(word, verbose=False):
    insert_l = []
    lower = string.ascii_lowercase
    for i in range(len(word)+1):
        temp = [word[0:i] + j + word[i:len(word)] for j in lower]
        insert_l.extend(temp)
    if verbose:
        print(f"Input word = {word}, insert_l = {insert_l}")
        print(f"len(insert_l) = {len(insert_l)}\n")
    return insert_l

In [25]:
def edit_1_letter(word):
    edit_1_set = set(delete_letter(word) + insert_letter(word) + replace_letter(word) + switch_letter(word))
    return edit_1_set

def edit_2_letters(word, allow_switches=True):
    edit_2_set = set()
    insert_letter1 = []
    replace_letter1 = []
    switch_letter1 = []
    delete_letter1 = []
    l = list(edit_1_letter(word))
    temp = []
    for i in l:
        temp = delete_letter(i)
        delete_letter1.extend(temp)
    for i in l:
        temp = replace_letter(i)
        replace_letter1.extend(temp)
    for i in l:
        temp = switch_letter(i)
        switch_letter1.extend(temp)
    for i in l:
        temp = insert_letter(i)
        insert_letter1.extend(temp)
    edit_2_set = set(replace_letter1 + switch_letter1 + delete_letter1 + insert_letter1)
    return edit_2_set

In [26]:
def get_corrections(word, probabilities, vocab, n=2, verbose=True):
    suggestions = []
    n_best = []
    if word in vocab:
        suggestions = word
    elif len(edit_1_letter(word)) != 0:
        suggestions = edit_1_letter(word).intersection(set(vocab))
    elif len(edit_2_letters(word)) != 0:
        suggestions = edit_2_letters(word).intersection(set(vocab))
    else:
        suggestions = word
    best_words = {i:probabilities[i] for i in suggestions if i in probabilities}
    best_words = [(k, v) for k, v in sorted(best_words.items(), key=lambda item: item[1], reverse=True)]
    n_best = best_words[:n]
    if verbose:
        print(f"Enter word : {word}, suggestions = {suggestions}")
    return n_best

In [27]:
def min_edit_distance(source, target, insert_cost=1, delete_cost=1, replace_cost=2):
    a, b, c = 0, 0, 0
    d = []
    len_src = len(source)
    len_target = len(target)
    Dimension = np.zeros((len_src+1, len_target+1), dtype=int)
    for row in range(0, len_src+1):
        Dimension[row, 0] = row
    for col in range(0, len_target+1):
        Dimension[0, col] = col
    for row in range(1, len_src+1):
        for col in range(1, len_target+1):
            r_cost = replace_cost
            if source[row-1] == target[col-1]:
                r_cost = 0
            a = Dimension[row-1, col] + delete_cost
            b = Dimension[row, col-1] + insert_cost
            c = Dimension[row-1, col-1] + r_cost
            d = [a, b, c]
            Dimension[row, col] = min(d)
    minimum_edit_distance = Dimension[len_src, len_target]
    return Dimension, minimum_edit_distance

In [28]:
def similarity(word, word_l):
    return [1-(textdistance.Jaccard(qval=2).distance(v,word)) for v in word_l]

In [29]:
def summary(word, word_l, sim,  probs, min_edit=[]):
    df = pd.DataFrame.from_dict(probs, orient='index').reset_index()
    df = df.rename(columns={'index':'Word', 0:'Prob'})
    df['Word'] = word_l
    df['Similarity'] = sim
    if min_edit:
        df['Min Edit'] = min_edit
    if not len(min_edit):
        output = df.sort_values(['Similarity'], ascending=False).head()
    else:
        output = df.sort_values(['Min Edit', 'Similarity'], ascending=[True, False]).head(10)
    return output

In [30]:
if word in word_l:
    print(f"We have {word} in our dictionary.")
else:
    matrix, min_edit, df = [], [], []
    for i in range(len(word_l)):
        matrix_temp, min_edit_temp = min_edit_distance(word, word_l[i])
        matrix.append(matrix_temp)
        min_edit.append(min_edit_temp)
        idx = list("#" + word)
        cols = list("#" + word_l[i])
        df_temp = pd.DataFrame(matrix_temp, index=idx, columns=cols)
        df.append(df_temp)
    
    sim = similarity(word, word_l)
    for i in range(len(min_edit)):
        for j in range(len(min_edit)-1):
            if min_edit[j] > min_edit[j+1]:
                min_edit[j], min_edit[j+1] = min_edit[j+1], min_edit[j]
                word_l[j], word_l[j+1] = word_l[j+1], word_l[j]
                sim[j], sim[j+1] = sim[j+1], sim[j]
                df[j], df[j+1] = df[j+1], df[j]

    count = 0
    limits = 10
    for i in range(50):
        if count == limits:
            break
        for j in range(len(word_l)):
            if count == limits:
                break
            if i == min_edit[j]:
                print(f"\nword {j}: {word_l[j]}")
                print(f"minimum edits = {min_edit[j]}\n")
                print(df[j])
                print("-"*50)
                count += 1
    
    print(summary(word, word_l, sim, probabilities, min_edit))


word 0: linear
minimum edits = 2

   #  l  i  n  e  a  r
#  0  1  2  3  4  5  6
l  1  0  1  2  3  4  5
i  2  1  0  1  2  3  4
n  3  2  1  0  1  2  3
a  4  3  2  1  2  1  2
a  5  4  3  2  3  2  3
r  6  5  4  3  4  3  2
--------------------------------------------------

word 1: linda
minimum edits = 3

   #  l  i  n  d  a
#  0  1  2  3  4  5
l  1  0  1  2  3  4
i  2  1  0  1  2  3
n  3  2  1  0  1  2
a  4  3  2  1  2  1
a  5  4  3  2  3  2
r  6  5  4  3  4  3
--------------------------------------------------

word 2: in
minimum edits = 4

   #  i  n
#  0  1  2
l  1  2  3
i  2  1  2
n  3  2  1
a  4  3  2
a  5  4  3
r  6  5  4
--------------------------------------------------

word 3: line
minimum edits = 4

   #  l  i  n  e
#  0  1  2  3  4
l  1  0  1  2  3
i  2  1  0  1  2
n  3  2  1  0  1
a  4  3  2  1  2
a  5  4  3  2  3
r  6  5  4  3  4
--------------------------------------------------

word 4: link
minimum edits = 4

   #  l  i  n  k
#  0  1  2  3  4
l  1  0  1  2  3
i  2  1  0 